In [16]:
pip install openmeteo_requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install requests-cache retry-requests numpy pandas


Defaulting to user installation because normal site-packages is not writeable
  Using cached requests_cache-1.2.0-py3-none-any.whl (61 kB)
  Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Using cached cattrs-23.2.3-py3-none-any.whl (57 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 KB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://api.open-meteo.com/v1/forecast"

In [21]:
def apiOpenMeteo(latitude, longitude, cidade):
	params = {
		"latitude": latitude,
		"longitude": longitude,
		"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "pressure_msl", "cloud_cover", "visibility", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
		"timezone": "America/Sao_Paulo",
		"past_days": 92
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Cidade: {cidade}")

	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
	hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
	hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
	hourly_precipitation_probability = hourly.Variables(4).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
	hourly_rain = hourly.Variables(6).ValuesAsNumpy()
	hourly_showers = hourly.Variables(7).ValuesAsNumpy()
	hourly_snowfall = hourly.Variables(8).ValuesAsNumpy()
	hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
	hourly_cloud_cover = hourly.Variables(10).ValuesAsNumpy()
	hourly_visibility = hourly.Variables(11).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(12).ValuesAsNumpy()
	hourly_wind_direction_10m = hourly.Variables(13).ValuesAsNumpy()
	hourly_wind_gusts_10m = hourly.Variables(14).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
	hourly_data["dew_point_2m"] = hourly_dew_point_2m
	hourly_data["apparent_temperature"] = hourly_apparent_temperature
	hourly_data["precipitation_probability"] = hourly_precipitation_probability
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["rain"] = hourly_rain
	hourly_data["showers"] = hourly_showers
	hourly_data["snowfall"] = hourly_snowfall
	hourly_data["pressure_msl"] = hourly_pressure_msl
	hourly_data["cloud_cover"] = hourly_cloud_cover
	hourly_data["visibility"] = hourly_visibility
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
	hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
	hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

	hourly_dataframe = pd.DataFrame(data = hourly_data)
	hourly_dataframe["cidade"] = cidade
	
	return hourly_dataframe 

In [20]:
capitaisBr = [
    {"capital": "Maceió", "latitude": -8.1756, "longitude": -34.8631},
    {"capital": "São Paulo", "latitude": -23.5522, "longitude": -46.6333},
    {"capital": "Goiânia", "latitude": -16.6892, "longitude": -49.2786},
    {"capital": "Florianópolis", "latitude": -27.5945, "longitude": -48.5669},
    {"capital": "Belo Horizonte", "latitude": -19.9211, "longitude": -47.8333},
    {"capital": "Manaus", "latitude": -3.1056, "longitude": -60.0250},
    {"capital": "Salvador", "latitude": -12.9792, "longitude": -38.4750},
    {"capital": "Porto Velho", "latitude": -8.7792, "longitude": -62.7883},
    {"capital": "Rio de Janeiro", "latitude": -22.8833, "longitude": -43.2333},
    {"capital": "Natal", "latitude": -5.7950, "longitude": -35.7100},
    {"capital": "Curitiba", "latitude": -25.4272, "longitude": -49.6931},
    {"capital": "Recife", "latitude": -8.0500, "longitude": -34.9417},
    {"capital": "Cuiabá", "latitude": -15.6008, "longitude": -57.9172},
    {"capital": "Porto Alegre", "latitude": -30.0333, "longitude": -51.2000},
    {"capital": "Aracaju", "latitude": -9.9211, "longitude": -37.9250},
    {"capital": "Rio Branco", "latitude": -9.9750, "longitude": -67.8333},
    {"capital": "Belém", "latitude": -1.4000, "longitude": -48.4833},
    {"capital": "João Pessoa", "latitude": -7.9667, "longitude": -34.8750},
    {"capital": "Teresina", "latitude": -5.1050, "longitude": -42.8167},
    {"capital": "Palmas", "latitude": -10.2750, "longitude": -48.3333},
    {"capital": "Vitória", "latitude": -20.3122, "longitude": -40.3333},
    {"capital": "Fortaleza", "latitude": -3.7172, "longitude": -38.5431},
    {"capital": "São Luís", "latitude": -2.5100, "longitude": -44.3250},
    {"capital": "Campo Grande", "latitude": -20.4500, "longitude": -54.6083},
    {"capital": "Boa Vista", "latitude": -4.5000, "longitude": -66.9250},
    {"capital": "Maceió", "latitude": -8.1756, "longitude": -34.8631},
    {"capital": "Belo Horizonte", "latitude": -19.9211, "longitude": -47.8333},
    {"capital": "Curitiba", "latitude": -25.4272, "longitude": -49.6931},
    {"capital": "Florianópolis", "latitude": -27.5945, "longitude": -48.5669},
    {"capital": "Porto Alegre", "latitude": -30.0333, "longitude": -51.2000},
    {"capital": "Aracaju", "latitude": -9.92, "longitude": -37}
]


In [26]:
df = pd.DataFrame()
for elem in capitaisBr:
    dfTemp = apiOpenMeteo(elem["latitude"], elem["longitude"], elem["capital"])
    df = pd.concat([df, dfTemp], axis=0, ignore_index=True)


Coordinates -8.125°N -34.875°E
Cidade: Maceió
Coordinates -23.5°N -46.625°E
Cidade: São Paulo
Coordinates -16.75°N -49.25°E
Cidade: Goiânia
Coordinates -27.625°N -48.5°E
Cidade: Florianópolis
Coordinates -19.875°N -47.875°E
Cidade: Belo Horizonte
Coordinates -3.125°N -60.0°E
Cidade: Manaus
Coordinates -12.875°N -38.5°E
Cidade: Salvador
Coordinates -8.75°N -62.75°E
Cidade: Porto Velho
Coordinates -22.875°N -43.25°E
Cidade: Rio de Janeiro
Coordinates -5.75°N -35.75°E
Cidade: Natal
Coordinates -25.375°N -49.75°E
Cidade: Curitiba
Coordinates -8.0°N -35.0°E
Cidade: Recife
Coordinates -15.625°N -57.875°E
Cidade: Cuiabá
Coordinates -30.0°N -51.25°E
Cidade: Porto Alegre
Coordinates -9.875°N -37.875°E
Cidade: Aracaju
Coordinates -10.0°N -67.875°E
Cidade: Rio Branco
Coordinates -1.375°N -48.5°E
Cidade: Belém
Coordinates -8.0°N -34.875°E
Cidade: João Pessoa
Coordinates -5.125°N -42.875°E
Cidade: Teresina
Coordinates -10.25°N -48.375°E
Cidade: Palmas
Coordinates -20.125°N -40.5°E
Cidade: Vitória
C

In [32]:
df.loc[df["cidade"] == "Recife"]

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,pressure_msl,cloud_cover,visibility,wind_speed_10m,wind_direction_10m,wind_gusts_10m,cidade
26136,2024-02-20 03:00:00+00:00,26.799500,90.0,25.020367,33.334385,13.0,0.2,0.0,0.2,0.0,1010.500000,100.0,24140.0,1.484318,194.036270,6.480000,Recife
26137,2024-02-20 04:00:00+00:00,26.699499,91.0,25.107027,33.201855,16.0,0.0,0.0,0.0,0.0,1010.200012,98.0,24140.0,2.099143,210.963684,6.840000,Recife
26138,2024-02-20 05:00:00+00:00,26.499500,92.0,25.092693,33.132854,20.0,0.0,0.0,0.0,0.0,1009.599976,67.0,24140.0,1.138420,198.435043,4.320000,Recife
26139,2024-02-20 06:00:00+00:00,26.249500,93.0,25.026527,32.839783,23.0,0.0,0.0,0.0,0.0,1009.099976,67.0,24140.0,1.138420,161.564957,2.880000,Recife
26140,2024-02-20 07:00:00+00:00,26.199499,92.0,24.795811,32.639397,45.0,0.0,0.0,0.0,0.0,1009.299988,50.0,24140.0,1.138420,198.435043,2.880000,Recife
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2024-05-28 22:00:00+00:00,26.149500,84.0,23.232025,31.081814,28.0,0.0,0.0,0.0,0.0,1014.700012,37.0,24140.0,4.802999,167.005386,16.199999,Recife
28508,2024-05-28 23:00:00+00:00,25.899500,85.0,23.183079,30.925922,27.0,0.0,0.0,0.0,0.0,1015.400024,47.0,24140.0,3.960000,180.000000,12.240000,Recife
28509,2024-05-29 00:00:00+00:00,25.699499,87.0,23.371883,30.889099,26.0,0.0,0.0,0.0,0.0,1015.799988,56.0,24140.0,3.617955,185.710495,8.640000,Recife
28510,2024-05-29 01:00:00+00:00,25.549500,88.0,23.413759,30.816742,26.0,0.0,0.0,0.0,0.0,1015.799988,55.0,23700.0,3.259938,186.340103,8.280000,Recife
